## Field Check (MMA instead already)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dh = 4.4
dv = 1.0
Bh = 8.0
Bv = 25.0
xv = -5.0
yv = 0.0
zv = -1.0*dv
xh = 0
yh = 0
zh = -1*dh

xmin = -12; xmax = 8; ymin = -10; ymax = 10; zmin = 0; zmax = 1
n1 = 320; n2 = 320; n3 = 16


def wyper2016a_Avec(r):
    """Calculate vector potential A at position vector r=(x,y,z)"""
    x, y, z = r[...,0], r[...,1], r[...,2]
    
    # Pre-calculate common terms
    lv = ((x-xv)**2 + (y-yv)**2 + (z-zv)**2)**(-3/2)
    lh = ((x-xh)**2 + (y-yh)**2 + (z-zh)**2)**(-3/2)
    
    ch = (Bh*dh**3)/2
    cv = (Bv*dv**3)/2
    
    # Calculate vector potential components
    Ax = -cv*(y-yv)*lv
    Ay = cv*(x-xv)*lv -ch*(z-zh)*lh
    Az =  ch*(y-yh)*lh
    
    return np.stack([Ax, Ay, Az], axis=-1)

def wyper2016a_Bvec(r):
    """Calculate vector potential A at position vector r=(x,y,z)"""
    x, y, z = r[...,0], r[...,1], r[...,2]
    ch = Bh*dh**3/2
    cv = Bv*dv**3/2
    
    Bvec = np.zeros_like(r)
    Bvec[...,0] = -3*ch*((z-zh)**2+(y-yh)**2)/((x-xh)**2+(y-yh)**2+(z-zh)**2)**(5/2) + \
                   3*cv*(x-xv)*(z-zv)/((x-xv)**2+(y-yv)**2+(z-zv)**2)**(5/2) + \
                   2*ch/((x-xh)**2+(y-yh)**2+(z-zh)**2)**(3/2)

    Bvec[...,1] = 3*ch*(x-xh)*(y-yh)/((x-xh)**2+(y-yh)**2+(z-zh)**2)**(5/2) + \
                  3*cv*(y-yv)*(z-zv)/((x-xv)**2+(y-yv)**2+(z-zv)**2)**(5/2)

    Bvec[...,2] = 3*ch*(x-xh)*(z-zh)/((x-xh)**2+(y-yh)**2+(z-zh)**2)**(5/2) + \
                  -3*cv*((y-yv)**2+(x-xv)**2)/((x-xv)**2+(y-yv)**2+(z-zv)**2)**(5/2) + \
                  2*cv/((x-xv)**2+(y-yv)**2+(z-zv)**2)**(3/2)
    return Bvec

# Create 3D grid of points
x = np.linspace(xmin, xmax, n1)
y = np.linspace(ymin, ymax, n2)
z = np.linspace(zmin, zmax, n3)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Combine into position vectors
r = np.stack([X, Y, Z], axis=-1)


In [ ]:

# Calculate vector potential field
Avec = wyper2016a_Avec(r)

# Calculate curl B = ∇ × A
dx = x[1] - x[0]
dy = y[1] - y[0] 
dz = z[1] - z[0]

# Initialize arrays with same size as input
Bx = np.zeros_like(X)
By = np.zeros_like(Y)
Bz = np.zeros_like(Z)

# Central difference for interior points
# Note: all differences need to be taken over the same index ranges
Bx[1:-1,1:-1,1:-1] = (Avec[1:-1,2:,1:-1,2] - Avec[1:-1,:-2,1:-1,2])/(2*dy) - \
                      (Avec[1:-1,1:-1,2:,1] - Avec[1:-1,1:-1,:-2,1])/(2*dz)
By[1:-1,1:-1,1:-1] = (Avec[1:-1,1:-1,2:,0] - Avec[1:-1,1:-1,:-2,0])/(2*dz) - \
                      (Avec[2:,1:-1,1:-1,2] - Avec[:-2,1:-1,1:-1,2])/(2*dx)
Bz[1:-1,1:-1,1:-1] = (Avec[2:,1:-1,1:-1,1] - Avec[:-2,1:-1,1:-1,1])/(2*dx) - \
                      (Avec[1:-1,2:,1:-1,0] - Avec[1:-1,:-2,1:-1,0])/(2*dy)

# Combine into B field array
B = np.stack([Bx, By, Bz], axis=-1)
# Calculate B field directly using analytical expressions
Bvec = wyper2016a_Bvec(r)

In [ ]:
plt.imshow(Bvec[:,:,0,2])

## check the stored bottom b3 

In [ ]:
import numpy as np

def read_fortran_binary(filename):
    with open(filename, 'rb') as f:
        # Read the dimensions (two integers)
        nx1, nx2 = np.fromfile(f, dtype=np.int32, count=2)
        
        # Read the array data (double precision)
        data = np.fromfile(f, dtype=np.float64)
        
        # Reshape the array to match the original dimensions
        data = data.reshape((nx2, nx1))
        
    return data

# Read the file
b3e = read_fortran_binary('./data/b3g.bin')
jpara = b3e[1:,:]-b3e[:-1,:]
jperp = b3e[:,1:]-b3e[:,:-1]

# Display the array
print("Array shape:", b3e.shape)

# If you want to visualize it
import matplotlib.pyplot as plt

plt.imshow(b3e, cmap='viridis',extent=[-10, 5, -6, 6])
plt.colorbar()
plt.title('b3e Array Visualization')
plt.show()

In [ ]:
dh = 4.4; Bh = 8.0; dv = 1.0; Bv = 25.0; xv = -5.0
ch = (Bh*dh**3.0)*1/2; cv = (Bv*dv**3.0)*1/2

def wyper2016a_b3(xc1, xc2, xc3, dh, dv, ch, cv, xv):
    """Calculate B3 component of magnetic field.
    
    Args:
        xc1, xc2, xc3: Position coordinates
        dh, dv: Depth parameters
        ch, cv: Field strength parameters 
        xv: x-position of vertical dipole
        
    Returns:
        b3: B3 component of magnetic field
    """
    # Horizontal dipole position
    xh = 0.0
    yh = 0.0  
    zh = -1 * dh

    # Vertical dipole position (xv from parameters)
    yv = 0.0
    zv = -1 * dv

    # Calculate B3 component
    b3 = (3.0*ch*(xc1-xh)*(xc3-zh) / ((xc1-xh)**2 + (xc2-yh)**2 + (xc3-zh)**2)**(5/2) + 
          -3.0*cv*((xc2-yv)**2 + (xc1-xv)**2) / ((xc1-xv)**2 + (xc2-yv)**2 + (xc3-zv)**2)**(5/2) + 
          2.0*cv/((xc1-xv)**2 + (xc2-yv)**2 + (xc3-zv)**2)**(3/2))
    
    return b3

nx1 = 240; nx2 = 144
b3 = np.zeros((nx1, nx2))
dx1 = 20/240; dx2 = 12/144
xmin1 = -12; xmax1 = 8; xmin2 = -6; xmax2 = 6
for i in range(nx1):
    xi = xmin1 + i*dx1
    for j in range(nx2):
        xj = xmin2 + j*dx2
        b3[i,j] = wyper2016a_b3(xi, xj, 0.0, dh, dv, ch, cv, xv)

plt.imshow(b3.T, cmap='Greys', extent=[xmin1, xmax1, xmin2, xmax2])
plt.colorbar()
plt.show()

## .dat file check

In [ ]:
import simesh
import numpy as np
import matplotlib.pyplot as plt
ds = simesh.amr_loader('./data/stitch0000.dat')
ds.update()

In [ ]:
from matplotlib.colors import LogNorm

xmin = [-12, -10, 0.3]; xmax = [8, 10, 0.4]; nx = 500; ny = 500; nz = 1
datau = ds.mesh.export_slab_uniform(xmin, xmax, nx, ny, nz)

plt.imshow(np.rot90(datau[:,:,0,1]), cmap='viridis')
plt.colorbar()

In [ ]:
datau[250,250,0,:]

In [ ]:
datau.shape

## debug with slice datfile

In [ ]:
import yt
import glob
import os

from simesh.frontends.amrvac.datio import get_header

data_slice = './data'
slice_list = glob.glob(os.path.join(data_slice, 'stitch_d2*.dat'))
slice_list.sort()

units = dict(length_unit=(1e9, 'cm'), temperature_unit=(1e6, 'K'),
            numberdensity_unit=(1e9, 'cm**-3'))

In [ ]:
slice_list = glob.glob('./data/stitch_d1*.dat')
slice_list.sort()
slice0 = slice_list[0]
# slice0 = './data/stitch_d2_x+0.00D+00_n0000.dat'
ds = yt.load(slice0, units_override=units, unit_system='cgs')
phys_vars = ['rho']
for var in phys_vars:
    p = yt.plot_2d(ds, var, origin='native')
    # p.set_zlim(var, 1e-1, 1e2)
    # p.set_zlim(var, 0.1, 10)
    p.annotate_timestamp()      
    p.annotate_grids()
    # Handle negative values when setting log scale
    if (ds.r[var] < 0).any():
        p.set_log(var, False)  # Fall back to linear scale if negative values present
    else:
        p.set_log(var, True)
    p.set_log(var, False)  # Fall back to linear scale if negative values present
    p.annotate_streamlines('b2','b3', density=5, color='white', linewidth=1.2)
    p.annotate_grids()
    p.show()

In [ ]:
slice_list = glob.glob('./datas/stitch_d2*.dat')
slice_list.sort()
for i in range(len(slice_list)):
    if i % 2 != 0:
        continue
    slice0 = slice_list[i]
    ds = yt.load(slice0, units_override=units, unit_system='cgs')
    phys_vars = ['rho']
    for var in phys_vars:
        p = yt.plot_2d(ds, var, origin='native')
        # p.set_zlim(var, -0.4, 1.2)
        # p.set_zlim(var, 1e-1, 1e2)
        p.annotate_timestamp()
        p.annotate_grids()
        if (ds.r[var] <= 0).any():
            p.set_log(var, False)  # Fall back to linear scale if negative values present
        else:
            p.set_log(var, True)
        p.set_log(var, False)  # Fall back to linear scale if negative values present
        p.annotate_streamlines('b1','b3', density=3, color='white', linewidth=1.2)
        p.save('./figs/slice'+str(i).zfill(4)+'_'+var+'.png')

# ffmpeg -framerate 12 -pattern_type glob -i ./figs/'slice*_rho.png' -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p rho2.mp4 

### Direct analysis on 